# Retail weekly report

## *Version : Alpha 01*

1) Collecte des données
* Chargement des fichiers 
* Formattage en dataframe : zones et point de passage
* Création des données complémentaires (jours, numero de semaines, année)

2) Traitement
* Indicateurs globaux
* Comparatifs sites
* Donnée site de référence

3) Génération du rapport
* Indicateurs globaux
* Comparatifs sites
* Données par site

# 0) Initialisation

* Mise en place des librairies standard
* Chargement des modules utilisateurs
* Définition des variables globales

In [ ]:
# Imports

import pandas as pd
import xlsxwriter as xw

from datetime import datetime

In [ ]:
# Key constants

WEEK_DAYS = ['Lundi','Mardi','Mercredi','Jeudi','Vendredi','Samedi','Dimanche']
WEEK_DAYS_ABBR = ['Lun','Mar','Mer','Jeu','Ven','Sam','Dim']

RAW_DATA_PATH = "raw_data/"
MEDIA_PATH = "media/"
OUTPUT_PATH = "xlsx_output/"

ZONE_RAW_FILE = "Carrefour_hyper_KPIs_1-March-2021_to_1-June-2021_zone.xlsx"
PASSAGE_RAW_FILE = "Carrefour_hyper_KPIs_1-March-2021_to_1-June-2021_passage.xlsx"

# 1) Collecte des données

Prérequis :
* Periode bien définie, commençant par un lundi et finissant par un dimanche, au moins deux semaines
* Contient tous les magasins du réseau à analyser

Fichiers :
* KPI sur periode par passage.xlsx
* KPI sur periode par zone.xlsx

Erreurs possibles :
* Non prise en compte d'erreurs de capture d'information / d'informations manquantes


In [27]:
# load passage & zone
df_passage_raw = pd.read_excel(RAW_DATA_PATH + PASSAGE_RAW_FILE) 
df_zone_raw = pd.read_excel(RAW_DATA_PATH + ZONE_RAW_FILE) 


**TRAITEMENT SPECIAL 1 : renommer la colonne Date.1 en Hour**

In [34]:
# Date.1 -> Hour dans le titre de colonne

df_passage_raw.rename(columns = {'Date.1': 'Hour'}, inplace = True)

**TRAITEMENT SPECIAL : correction des fichiers démarrants à :40 ... (décalage vers 8:00 -> données fausses)**

In [32]:
# corrige / normalise les décalages horaires - attention pas normal d'avoir des slots qui commencent :40 ...
df_passage_raw['Hour'] = df_passage_raw['Hour'].str.replace(':40',':00')
df_zone_raw['Hour'] = df_zone_raw['Hour'].str.replace(':40',':00')